In [1]:
import os 
import random
import shutil
from pathlib import Path
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch
from torch import nn 
from tqdm.auto import tqdm

/home/mahdi/Desktop/emotion/emo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def count_images_in_folders(base_dir):
    for split in os.listdir(base_dir):
            split_path = os.path.join(base_dir, split)
            if os.path.isdir(split_path):
                  print(f"\n Split: {split}")
                  for emotion in os.listdir(split_path):
                        emotion_path = os.path.join(split_path,emotion)
                        if os.path.isdir(emotion_path):
                              count = len([
                                    f for f in os.listdir(emotion_path)
                                    if os.path.isfile(os.path.join(emotion_path,f))
                              ])
                              print(f" {emotion}: {count} images")

In [3]:
count_images_in_folders("fer2013_data")  


 Split: val
 fear: 409 images
 surprise: 317 images
 sad: 483 images
 happy: 721 images
 disgust: 43 images
 neutral: 496 images
 angry: 399 images

 Split: train
 fear: 3688 images
 surprise: 2854 images
 sad: 4347 images
 happy: 6494 images
 disgust: 393 images
 neutral: 4469 images
 angry: 3596 images

 Split: test
 fear: 1024 images
 surprise: 831 images
 sad: 1247 images
 happy: 1774 images
 disgust: 111 images
 neutral: 1233 images
 angry: 958 images


In [4]:
def split_train_val(train_dir, val_dir, val_ratio = 0.1, seed = 42):
    random.seed(seed)

    Path(val_dir).mkdir(parents=True, exist_ok=True)

    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir,class_name)
        if not os.path.isdir(class_path):
            continue
        
        images = os.listdir(class_path)
        random.shuffle(images)

        n_val = int(len(images) * val_ratio)
        val_images = images[:n_val]

        val_class_path = os.path.join(val_dir, class_name)
        Path(val_class_path).mkdir(parents=True, exist_ok= True)

        for img_name in val_images:
            src_path = os.path.join(class_path,img_name)
            dst_path = os.path.join(val_class_path, img_name)
            shutil.move(src_path,dst_path)

        print(f"{class_name}: moved {n_val} images to validation set")

In [5]:
# split_train_val(
#     train_dir='fer2013_data/train',
#     val_dir='fer2013_data/val',
#     val_ratio=0.1,
# )

In [6]:
image_size = 48

train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Grayscale(num_output_channels=1), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])

])

val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Grayscale(num_output_channels=1),  
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])

])

train_dataset = datasets.ImageFolder('fer2013_data/train', transform=train_transform)
val_dataset = datasets.ImageFolder('fer2013_data/val', transform=val_test_transform)
test_dataset = datasets.ImageFolder('fer2013_data/test', transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [7]:
train_dataset.class_to_idx

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [8]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using: {device} device")

Using: cuda device


In [9]:
class EmotionCNNV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1,16, 3),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(16,32,3),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(4096,7)
        )


    def forward(self,x):
        logits = self.layers(x)
        return logits


In [10]:
model = EmotionCNNV1().to(device)

In [11]:
dummy = torch.randn(1, 1, 48, 48)  # (batch, channel, H, W)
out = model.layers[:-2](dummy.to(device))  # Skip Flatten and Linear
print(out.shape)  # --> torch.Size([1, 64, 39, 39])


torch.Size([1, 64, 8, 8])


In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

In [13]:
num_epochs = 15
best_val_acc = 0 

In [14]:
for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0
    correct = 0 
    total = 0 

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits,labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(logits, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            preds = torch.argmax(logits, dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch {epoch+1}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # ===== Save Best Model =====
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print("✅ Best model saved!")


  7%|▋         | 1/15 [00:05<01:10,  5.04s/it]

Epoch 1, Train Acc: 0.3792, Val Acc: 0.4575
✅ Best model saved!


 13%|█▎        | 2/15 [00:09<01:04,  4.96s/it]

Epoch 2, Train Acc: 0.4713, Val Acc: 0.4791
✅ Best model saved!


 20%|██        | 3/15 [00:14<00:59,  4.97s/it]

...

Epoch 13, Train Acc: 0.6418, Val Acc: 0.5520


 93%|█████████▎| 14/15 [01:09<00:04,  4.90s/it]

Epoch 14, Train Acc: 0.6509, Val Acc: 0.5586


100%|██████████| 15/15 [01:13<00:00,  4.93s/it]

Epoch 15, Train Acc: 0.6586, Val Acc: 0.5551


In [15]:
model.eval()
val_correct = 0
val_total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        preds = torch.argmax(logits, dim=1)
        val_correct += (preds == labels).sum().item()
        val_total += labels.size(0)

val_acc = val_correct / val_total
print(f"Val Acc: {val_acc:.4f}")


100%|██████████| 113/113 [00:01<00:00, 109.39it/s]

Val Acc: 0.5582
